In [1]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 12.7 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import math

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [ ]:
FILL_IN = "FILL_IN"

In [3]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-12-11 20:34:11--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z     [ <=>                ] 198.65K  --.-KB/s    in 0.1s    

2024-12-11 20:34:11 (1.47 MB/s) - ‘smsspamcollection.zip’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [4]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [5]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [6]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-12-11 20:34:12--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.1’

smsspamcollection.z     [ <=>                ] 198.65K  --.-KB/s    in 0.1s    

2024-12-11 20:34:12 (1.84 MB/s) - ‘smsspamcollection.zip.1’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [7]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
text = df.text.values
labels = df.label.values

In [9]:
# Get the bert-base-uncased tokenizer and set lower case to True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ dear     │        6203 │
├──────────┼─────────────┤
│ dave     │        4913 │
├──────────┼─────────────┤
│ this     │        2023 │
├──────────┼─────────────┤
│ is       │        2003 │
├──────────┼─────────────┤
│ your     │        2115 │
├──────────┼─────────────┤
│ final    │        2345 │
├──────────┼─────────────┤
│ notice   │        5060 │
├──────────┼─────────────┤
│ to       │        2000 │
├──────────┼─────────────┤
│ collect  │        8145 │
├──────────┼─────────────┤
│ your     │        2115 │
├──────────┼─────────────┤
│ 4        │        1018 │
├──────────┼─────────────┤
│ *        │        1008 │
├──────────┼─────────────┤
│ ten      │        2702 │
├──────────┼─────────────┤
│ ##eri    │       11124 │
├──────────┼─────────────┤
│ ##fe     │        7959 │
├──────────┼─────────────┤
│ holiday  │        6209 │
├──────────┼─────────────┤
│ or       │        2030 │
├──────────┼─────────────┤
│

In [11]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=32,
        padding='max_length',
        return_tensors='pt',
        truncation=True,
        return_attention_mask=True
    )


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

In [12]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

╒══════════╤═════════════╤══════════════════╕
│ Tokens   │   Token IDs │   Attention Mask │
╞══════════╪═════════════╪══════════════════╡
│ [CLS]    │         101 │                1 │
├──────────┼─────────────┼──────────────────┤
│ not      │        2025 │                1 │
├──────────┼─────────────┼──────────────────┤
│ much     │        2172 │                1 │
├──────────┼─────────────┼──────────────────┤
│ no       │        2053 │                1 │
├──────────┼─────────────┼──────────────────┤
│ fights   │        9590 │                1 │
├──────────┼─────────────┼──────────────────┤
│ .        │        1012 │                1 │
├──────────┼─────────────┼──────────────────┤
│ it       │        2009 │                1 │
├──────────┼─────────────┼──────────────────┤
│ was      │        2001 │                1 │
├──────────┼─────────────┼──────────────────┤
│ a        │        1037 │                1 │
├──────────┼─────────────┼──────────────────┤
│ good     │        2204 │        

In [13]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    range(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(
    token_id[train_idx],
    attention_masks[train_idx],
    labels[train_idx]
)

val_set = TensorDataset(
    token_id[val_idx],
    attention_masks[val_idx],
    labels[val_idx]
)

# Prepare DataLoader
train_dataloader = train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

validation_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

Define the LoRA specific layers.

In [14]:
# Define a LoRA Layer which has A, B and alpha parameters
class LoRALayer(torch.nn.Module):
  def __init__(self, in_dim, out_dim, rank, alpha):
    super().__init__()
    # Initialize A to be a parameter matrix of dimension in_dim by rank
    # initaliza A as Gussian noise
    self.A = torch.nn.Parameter(torch.randn(in_dim, rank))
    # Initialize all the elements of A via kaiming_uniform with a equal to sqrt(5)
    torch.nn.init.kaiming_uniform_(self.A, a = math.sqrt(5))
    # Initialize B to be a zero parameter matrix of the appropriate dimensions
    self.B = torch.nn.Parameter(torch.zeros(rank, out_dim))
    self.alpha = alpha

  def forward(self, x):
    # Pass x through the LoRA layer and return the new x
    x = x @ self.A @ self.B * self.alpha
    return x

# Define a class LoRALinear which has a linear layer and a LoRA layer on top
class LoRALinear(torch.nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    self.linear = linear
    self.lora = LoRALayer(linear.in_features, linear.out_features, rank, alpha)

  def forward(self, x):
    # Pass x through the linear layer and also the lora layer
    return self.linear(x) + self.lora(x)

def lora_linear_replace(model, rank, alpha):
  # Use model.named_children to go through all layers and if the layer is Linear replace that layer with LoRALinear
  for name, module in model.named_children():
    # If the module is linear, replace the module in the model with a LoRA layer
    if isinstance(module, torch.nn.Linear):
      # Replace the Linear layer with LoRALinear
      setattr(model, name, LoRALinear(module, rank, alpha))
    else:
      # Alterntively, recursively apply the same function to child modules so that other Linear layers get replaced
      lora_linear_replace(module, rank, alpha)

### Load specific versions of the model

In [15]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)

# Turn off all gradients of the model to start
for param in model.parameters():
    param.requires_grad = False

# Set to True if LORA is used; if False, fine_tune flag will be used to decide if you fine tune the entire model or just parts
use_lora  = True
# If this is False, turn off gradients
fine_tune = False
# Set total_parameters to 0; this will count the number of parameters in each case
total_parameters = 0

if use_lora:
  # Use the lora_linear to attach a LoRA layer to each linear later of the original BERT model
  lora_linear_replace(model, rank = 8, alpha = 8)
  # Get the total number of parameters with gradients
  total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
else:
  # If fine_tune is off, turn off gradients for all layers other than classifier
  if not fine_tune:
    # Turn off all gradients; count just the 'classifier' layer which is the only one that has gradients
    for param in model.classifier.parameters():
      param.requires_grad = True
    total_parameters = sum(p.numel() for p in model.classifier.parameters() if p.requires_grad)
  else:
    # Turn off all gradients except for classifier
    for param in model.parameters():
      param.requires_grad = True
    total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(total_parameters)

if use_lora:
  assert(total_parameters == 1345552)
else:
  if fine_tune:
    assert(total_parameters == 109483778)
  else:
    assert(total_parameters == 1538)




model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1345552


### Set the model to the right device

In [16]:
import platform

# Pick the system you have and select GPU if you can
if platform.system() == 'Darwin':
    device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
elif platform.system() == 'Linux':
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
else:
    device = torch.device('cpu')
print(device)

cuda


In [17]:
_ = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [18]:
# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

### Train the model

In [19]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
import torchmetrics
accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=2).to(device)
recall = torchmetrics.Recall(task='multiclass', num_classes=2).to(device)
precision = torchmetrics.Precision(task='multiclass', num_classes=2).to(device)
auroc = torchmetrics.AUROC(task='multiclass', num_classes=2).to(device)

In [20]:
model.device

device(type='cuda', index=0)

In [21]:
# Main training / validation loop
import tqdm
for _ in tqdm.tqdm(trange(epochs, desc = 'Epoch')):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in tqdm.tqdm(enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        # Set the gradients to zero
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, attention_mask = b_input_mask, labels = b_labels)

        # Backward pass
        outputs.loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += outputs.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
            eval_output = model(b_input_ids, attention_mask = b_input_mask, labels = b_labels)

        # Calculate validation metrics
        labels = b_labels
        predicted_labels = torch.argmax(eval_output.logits, dim = -1)

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(eval_output.logits, labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1it [00:01,  1.71s/it]

2it [00:01,  1.29it/s]

3it [00:01,  2.12it/s]

5it [00:02,  4.05it/s]

7it [00:02,  6.04it/s]

9it [00:02,  7.91it/s]

11it [00:02,  9.47it/s]

13it [00:02, 10.84it/s]

15it [00:02, 11.96it/s]

17it [00:02, 12.73it/s]

19it [00:03, 13.43it/s]

21it [00:03, 13.91it/s]

23it [00:03, 14.33it/s]

25it [00:03, 14.34it/s]

27it [00:03, 14.43it/s]

29it [00:03, 14.65it/s]

31it [00:03, 14.77it/s]

33it [00:03, 14.68it/s]

35it [00:04, 14.72it/s]

37it [00:04, 14.86it/s]

39it [00:04, 14.97it/s]

41it [00:04, 14.82it/s]

43it [00:04, 14.97it/s]

45it [00:04, 15.03it/s]

47it [00:04, 14.86it/s]

49it [00:05, 14.91it/s]

51it [00:05, 14.99it/s]

53it [00:05, 15.08it/s]

55it [00:05, 14.93it/s]

57it [00:05, 14.87it/s]

59it [00:05, 14.96it/s]

61it [00:05, 14.82it/s]

63it [00:05, 14.98it/s]

65it [00:06, 14.95it/s]

67it [00:06, 15.07it/s]

69it [00:06, 15.03it/s]

71it [00:06, 14.79it/s]

73it [00:06, 15.00it/s


	 - Train loss: 0.0897
	 - Validation Accuracy: 0.9786
	 - Validation Precision: 0.9786
	 - Validation Recall: 0.9786
	 - Validation AUROC: 0.8857





0it [00:00, ?it/s]

2it [00:00, 15.91it/s]

4it [00:00, 15.44it/s]

6it [00:00, 15.33it/s]

8it [00:00, 15.19it/s]

10it [00:00, 14.97it/s]

12it [00:00, 15.02it/s]

14it [00:00, 15.00it/s]

16it [00:01, 15.09it/s]

18it [00:01, 15.13it/s]

20it [00:01, 15.07it/s]

22it [00:01, 15.06it/s]

24it [00:01, 14.95it/s]

26it [00:01, 13.82it/s]

28it [00:01, 14.02it/s]

30it [00:02, 14.36it/s]

32it [00:02, 14.60it/s]

34it [00:02, 14.57it/s]

36it [00:02, 14.64it/s]

38it [00:02, 14.79it/s]

40it [00:02, 14.66it/s]

42it [00:02, 14.76it/s]

44it [00:02, 14.82it/s]

46it [00:03, 14.80it/s]

48it [00:03, 14.85it/s]

50it [00:03, 14.91it/s]

52it [00:03, 14.92it/s]

54it [00:03, 14.69it/s]

56it [00:03, 14.52it/s]

58it [00:03, 14.73it/s]

60it [00:04, 14.76it/s]

62it [00:04, 14.91it/s]

64it [00:04, 14.89it/s]

66it [00:04, 14.91it/s]

68it [00:04, 14.93it/s]

70it [00:04, 14.43it/s]

72it [00:04, 14.66it/s]

74it [00:05, 14.65it/s]

76it [00:05, 14.80it/s]

78it [00:05, 14.74it/s]

80it [0


	 - Train loss: 0.0251
	 - Validation Accuracy: 0.9839
	 - Validation Precision: 0.9839
	 - Validation Recall: 0.9839
	 - Validation AUROC: 0.8819



### Test on a specific sentence, see the outcome

In [22]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


### Questions

Question 1: Run the above by fine tuning bert and the classfier head and by not doing this (using BERT as a feature encoder). What is the gap between this?

FILL_IN



In [23]:
# fine tuning (allow gradiet updates on the entire model)
model2 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)

# Turn off all gradients of the model to start
for param in model2.parameters():
    param.requires_grad = False

# Set to True if LORA is used; if False, fine_tune flag will be used to decide if you fine tune the entire model or just parts
use_lora  = False
# If this is False, turn off gradients
fine_tune = True
# Set total_parameters to 0; this will count the number of parameters in each case
total_parameters = 0

if use_lora:
  # Use the lora_linear to attach a LoRA layer to each linear later of the original BERT model
  lora_linear_replace(model2, rank = 8, alpha = 8)
  # Get the total number of parameters with gradients
  total_parameters = sum(p.numel() for p in model2.parameters() if p.requires_grad)
else:
  # If fine_tune is off, turn off gradients for all layers other than classifier
  if not fine_tune:
    # Turn off all gradients; count just the 'classifier' layer which is the only one that has gradients
    for param in model2.classifier.parameters():
      param.requires_grad = True
    total_parameters = sum(p.numel() for p in model2.classifier.parameters() if p.requires_grad)
  else:
    # Turn off all gradients except for classifier
    for param in model2.parameters():
      param.requires_grad = True
    total_parameters = sum(p.numel() for p in model2.parameters() if p.requires_grad)

print(total_parameters)

_ = model2.to(device)
epochs = 2
optimizer2 = torch.optim.AdamW(
    model2.parameters(),
    lr = 3e-5,
    eps = 1e-08
)

for _ in tqdm.tqdm(trange(epochs, desc = 'Epoch')):

    # ========== Training ==========

    # Set model to training mode
    model2.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in tqdm.tqdm(enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        # Set the gradients to zero
        model2.zero_grad()

        # Forward pass
        outputs = model2(b_input_ids, attention_mask = b_input_mask, labels = b_labels)

        # Backward pass
        outputs.loss.backward()
        optimizer2.step()

        # Update tracking variables
        tr_loss += outputs.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model2.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
            eval_output = model2(b_input_ids, attention_mask = b_input_mask, labels = b_labels)

        # Calculate validation metrics
        labels = b_labels
        predicted_labels = torch.argmax(eval_output.logits, dim = -1)

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(eval_output.logits, labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


109483778


  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1it [00:00,  6.08it/s]

2it [00:00,  6.29it/s]

3it [00:00,  7.33it/s]

4it [00:00,  7.89it/s]

5it [00:00,  8.22it/s]

6it [00:00,  8.45it/s]

7it [00:00,  8.63it/s]

8it [00:00,  8.74it/s]

9it [00:01,  8.83it/s]

10it [00:01,  8.91it/s]

11it [00:01,  8.91it/s]

12it [00:01,  8.97it/s]

13it [00:01,  8.95it/s]

14it [00:01,  8.99it/s]

15it [00:01,  9.02it/s]

16it [00:01,  9.05it/s]

17it [00:01,  9.07it/s]

18it [00:02,  9.03it/s]

19it [00:02,  9.05it/s]

20it [00:02,  9.03it/s]

21it [00:02,  9.10it/s]

22it [00:02,  9.06it/s]

23it [00:02,  9.04it/s]

24it [00:02,  9.00it/s]

25it [00:02,  9.02it/s]

26it [00:02,  9.02it/s]

27it [00:03,  8.97it/s]

28it [00:03,  8.97it/s]

29it [00:03,  8.99it/s]

30it [00:03,  8.97it/s]

31it [00:03,  8.99it/s]

32it [00:03,  9.00it/s]

33it [00:03,  8.98it/s]

34it [00:03,  8.96it/s]

35it [00:03,  8.95it/s]

36it [00:04,  8.97it/s]

37it [00:04,  9.01it/s]

38it [00:04,  9.07it/s]




	 - Train loss: 0.0864
	 - Validation Accuracy: 0.9848
	 - Validation Precision: 0.9848
	 - Validation Recall: 0.9848
	 - Validation AUROC: 0.8834





0it [00:00, ?it/s]

1it [00:00,  9.32it/s]

2it [00:00,  8.98it/s]

3it [00:00,  8.93it/s]

4it [00:00,  8.88it/s]

5it [00:00,  8.81it/s]

6it [00:00,  8.70it/s]

7it [00:00,  8.76it/s]

8it [00:00,  8.71it/s]

9it [00:01,  8.64it/s]

10it [00:01,  8.67it/s]

11it [00:01,  8.73it/s]

12it [00:01,  8.75it/s]

13it [00:01,  8.78it/s]

14it [00:01,  8.80it/s]

15it [00:01,  8.77it/s]

16it [00:01,  8.76it/s]

17it [00:01,  8.71it/s]

18it [00:02,  8.70it/s]

19it [00:02,  8.69it/s]

20it [00:02,  8.74it/s]

21it [00:02,  8.78it/s]

22it [00:02,  8.79it/s]

23it [00:02,  8.80it/s]

24it [00:02,  8.80it/s]

25it [00:02,  8.82it/s]

26it [00:02,  8.75it/s]

27it [00:03,  8.73it/s]

28it [00:03,  8.69it/s]

29it [00:03,  8.69it/s]

30it [00:03,  8.71it/s]

31it [00:03,  8.72it/s]

32it [00:03,  8.78it/s]

33it [00:03,  8.77it/s]

34it [00:03,  8.82it/s]

35it [00:03,  8.79it/s]

36it [00:04,  8.82it/s]

37it [00:04,  8.82it/s]

38it [00:04,  8.83it/s]

39it [00:04,  8.50it/s]

40it [00:04,


	 - Train loss: 0.0253
	 - Validation Accuracy: 0.9884
	 - Validation Precision: 0.9884
	 - Validation Recall: 0.9884
	 - Validation AUROC: 0.8783



In [25]:
# fine tuning just the classifier head
model3 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)

# Turn off all gradients of the model to start
for param in model3.parameters():
    param.requires_grad = False

# Set to True if LORA is used; if False, fine_tune flag will be used to decide if you fine tune the entire model or just parts
use_lora  = False
# If this is False, turn off gradients
fine_tune = False
# Set total_parameters to 0; this will count the number of parameters in each case
total_parameters = 0

if use_lora:
  # Use the lora_linear to attach a LoRA layer to each linear later of the original BERT model
  lora_linear_replace(model3, rank = 8, alpha = 8)
  # Get the total number of parameters with gradients
  total_parameters = sum(p.numel() for p in model3.parameters() if p.requires_grad)
else:
  # If fine_tune is off, turn off gradients for all layers other than classifier
  if not fine_tune:
    # Turn off all gradients; count just the 'classifier' layer which is the only one that has gradients
    for param in model3.classifier.parameters():
      param.requires_grad = True
    total_parameters = sum(p.numel() for p in model3.classifier.parameters() if p.requires_grad)
  else:
    # Turn off all gradients except for classifier
    for param in model3.parameters():
      param.requires_grad = True
    total_parameters = sum(p.numel() for p in model3.parameters() if p.requires_grad)

print(total_parameters)

_ = model3.to(device)
epochs = 2
optimizer3 = torch.optim.AdamW(
    model3.parameters(),
    lr = 3e-5,
    eps = 1e-08
)

for _ in tqdm.tqdm(trange(epochs, desc = 'Epoch')):

    # ========== Training ==========

    # Set model to training mode
    model3.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in tqdm.tqdm(enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        # Set the gradients to zero
        model3.zero_grad()

        # Forward pass
        outputs = model3(b_input_ids, attention_mask = b_input_mask, labels = b_labels)

        # Backward pass
        outputs.loss.backward()
        optimizer3.step()

        # Update tracking variables
        tr_loss += outputs.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model3.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
            eval_output = model3(b_input_ids, attention_mask = b_input_mask, labels = b_labels)

        # Calculate validation metrics
        labels = b_labels
        predicted_labels = torch.argmax(eval_output.logits, dim = -1)

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(eval_output.logits, labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1538


  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

3it [00:00, 23.68it/s]

6it [00:00, 25.15it/s]

9it [00:00, 26.61it/s]

13it [00:00, 29.35it/s]

17it [00:00, 30.72it/s]

21it [00:00, 31.43it/s]

25it [00:00, 30.86it/s]

29it [00:00, 30.23it/s]

33it [00:01, 31.00it/s]

37it [00:01, 31.67it/s]

41it [00:01, 32.03it/s]

45it [00:01, 32.29it/s]

49it [00:01, 32.52it/s]

53it [00:01, 32.77it/s]

57it [00:01, 31.80it/s]

61it [00:01, 31.39it/s]

65it [00:02, 31.93it/s]

69it [00:02, 32.52it/s]

73it [00:02, 32.67it/s]

77it [00:02, 32.58it/s]

81it [00:02, 33.02it/s]

85it [00:02, 33.02it/s]

89it [00:02, 32.47it/s]

93it [00:02, 32.40it/s]

97it [00:03, 32.40it/s]

101it [00:03, 31.59it/s]

105it [00:03, 31.99it/s]

109it [00:03, 32.24it/s]

113it [00:03, 32.16it/s]

117it [00:03, 32.40it/s]

121it [00:03, 32.40it/s]

125it [00:03, 32.44it/s]

129it [00:04, 32.22it/s]

133it [00:04, 32.43it/s]

137it [00:04, 32.25it/s]

141it [00:04, 32.49it/s]

145it [00:04, 32.67it/s]

149it [


	 - Train loss: 0.4615
	 - Validation Accuracy: 0.8670
	 - Validation Precision: 0.8670
	 - Validation Recall: 0.8670
	 - Validation AUROC: 0.3902





0it [00:00, ?it/s]

4it [00:00, 38.66it/s]

8it [00:00, 35.67it/s]

12it [00:00, 35.09it/s]

16it [00:00, 34.63it/s]

20it [00:00, 34.13it/s]

24it [00:00, 33.92it/s]

28it [00:00, 34.03it/s]

32it [00:00, 32.70it/s]

36it [00:01, 27.67it/s]

40it [00:01, 29.38it/s]

44it [00:01, 29.41it/s]

48it [00:01, 29.89it/s]

52it [00:01, 31.13it/s]

56it [00:01, 31.89it/s]

60it [00:01, 31.25it/s]

64it [00:02, 30.90it/s]

68it [00:02, 31.19it/s]

72it [00:02, 31.39it/s]

76it [00:02, 25.48it/s]

79it [00:02, 25.56it/s]

83it [00:02, 27.69it/s]

86it [00:02, 27.72it/s]

90it [00:02, 29.47it/s]

94it [00:03, 30.64it/s]

98it [00:03, 31.49it/s]

102it [00:03, 31.35it/s]

106it [00:03, 31.63it/s]

110it [00:03, 31.44it/s]

114it [00:03, 30.88it/s]

118it [00:03, 31.45it/s]

122it [00:03, 32.00it/s]

126it [00:04, 32.29it/s]

130it [00:04, 32.63it/s]

134it [00:04, 32.32it/s]

138it [00:04, 32.06it/s]

142it [00:04, 31.81it/s]

146it [00:04, 27.38it/s]

149it [00:04, 24.61it/s]

152it [00:05, 22.


	 - Train loss: 0.3952
	 - Validation Accuracy: 0.8670
	 - Validation Precision: 0.8670
	 - Validation Recall: 0.8670
	 - Validation AUROC: 0.6596



| **Metric**               | **LoRA**         | **Fine-Tuning All**  | **Fine-Tuning Classifier Head** |
|--------------------------|------------------|----------------------|---------------------------------|
| **Validation Accuracy**   | 98.39%           | 98.84%               | 86.70%                          |
| **Validation Precision**  | 98.39%           | 98.84%               | 86.70%                          |
| **Validation Recall**     | 98.38%           | 98.84%               | 86.70%                          |
| **Validation AUROC**      | 88.19%           | 87.83%               | 65.96%                          |


The results shows that using LoRA allows the model to achieve performance similar to fine-tuning all parameters, despite having far fewer parameters. While the fine-tuning classifier head only results in much worse performance because the rest of the model remains frozen and cannot learn task-specific representations.